In [3]:
import pandas as pd

import random
import re
import pickle
import nltk

In [36]:
dataset = pd.read_csv("reviews.csv",sep="\t")
#dataset.drop(columns= "Unnamed: 0",inplace = True)
dataset.head(3)

/home/anubhav/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,IP Address
0,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,217304173,BlackVue DR600GW-PMP,Mobile_Electronics,5,0.0,0.0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31,211.200.63.206
1,US,40835037,R31LOQ8JGLPRLK,B00OQMFG1Q,137313254,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5,0.0,1.0,N,Y,five star,it's great,2015-08-31,192.17.224.99
2,US,51469641,R2Y0MM9YE6OP3P,B00QERR5CY,82850235,iXCC Multi pack Lightning cable,Mobile_Electronics,5,0.0,0.0,N,Y,great cables,These work great and fit my life proof case fo...,2015-08-31,198.9.2.161


In [6]:
### WORKING ON TWITTER DATA FOR SENTIMENTAL ANALYSIS

with open("Pickle Files/classifier.pickle","rb") as f:
    clf = pickle.load(f)

with open("Pickle Files/TfidfModel.pickle","rb") as f:
    tfidf = pickle.load(f)

def getSentiment(text):

    # PREPROCESSING THE DATASET
    text = str(text)
    text = text.lower()
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"there's","there is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"it's","it is",text)
    text = re.sub(r"who's","who is",text)
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"they're","they are",text)
    text = re.sub(r"who're","who are",text)
    text = re.sub(r"ain't","am not",text)
    text = re.sub(r"wouldn't","would not",text)
    text = re.sub(r"shouldn't","should not",text)
    text = re.sub(r"can't","can not",text)
    text = re.sub(r"couldn't","could not",text)
    text = re.sub(r"won't","will not",text)
    
    text = re.sub(r"\W"," ",text)
    text = re.sub(r"\d"," ",text)
    text = re.sub(r"\s+[a-z]\s+"," ",text)
    text = re.sub(r"^[a-z]\s+"," ",text)    
    text = re.sub(r"\s+[a-z]$"," ",text)    
    text = re.sub(r"\s+"," ",text)    
    
    sent = clf.predict(tfidf.transform([text]).toarray())
    
    return sent[0]


## 1. different sentiment in review headline and review body

In [7]:
#1. different sentiment in review headline and review body

remove_reviews = []
# stores the list of review_id of fake reviews

for i in range(len(dataset)):
    #iterate through the whole dataset
    
        if( getSentiment( dataset["review_headline"][i] ) != getSentiment( dataset["review_body"][i] ) ):
            # checking if the sentiment of the body and the headline are not same
            
            remove_reviews.append(dataset["review_id"][i])
            # append review_id to the list of fake reviews.

In [9]:
len(remove_reviews)

32322

## 2. Users which are posting either all positive or negative reviews on different products of same brand


In [129]:
dataset = pd.read_csv("reviews2.csv",sep="\t")

/home/anubhav/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,4,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
#2. Users which are posting either all positive or negative reviews on different products of same brand

customers = dataset.groupby("customer_id")
# groups dataset by customers

customer_list = dataset["customer_id"].unique()
#list of unique customers

blocked_users = []
# it will store list users that are writing fake reviews

size = len(customer_list.tolist())
#size of total unique customers

for i in range(size):
    # iterate through all the customers
    
    brand_df = customers.get_group(customer_list[i])    
    # Dataframe for each customers
    
    brands = brand_df.groupby("product_parent")
    # groups reviews of each customers by brand
    
    brands_list = brand_df["product_parent"].unique()
    # unique list of brands for each customers reviews
    
    no_of_brands = len(brands_list.tolist())
    # no. of brands for which reviews had been written by the customer
    
    for j in range(no_of_brands):
        # iterate through all the brands
        
        product_df = brands.get_group(brands_list[j])
        # Dataframe of products for a brand for which a customer had written reviews
        
        no_of_products = len(product_df["product_id"])
        # no of products
        
        if no_of_products<=2:
            # it will filter the products which are less than 2 for a brand
            continue
            
        indices = product_df.index.values.tolist()
        # index of the dataframe of the products of each brand for each customers
        
        sentiment = getSentiment(product_df["review_body"][indices[0]])
        # sentiment of the review of the first product
        
        isSameSentiment = True
        
        for k in range(1,no_of_products):
            # iterate through all the products
            
            text = str(product_df["review_body"][indices[k]])
            # review of each product
            
            if getSentiment(text)!=sentiment :
                # if sentiment is different than discard it
                isSameSentiment = False
                break;
                
        if(isSameSentiment):
            # if sentiments of all the products of same brand by a customer is same, 
            #append customer_id to blocked users list
            
            remove_reviews.append(customer_list[i])
            break
        
    

In [12]:
len(remove_reviews)

32327

## 3.Reviews from same IP either all positive or negative reviews on different products of same brand

In [15]:
dataset = pd.read_csv("reviews2.csv",sep="\t")

In [16]:
#3.Reviews from same IP either all positive or negative reviews on different products of same brand


ip = dataset.groupby("IP Address")
#grouping the dataset by ip address

ip_list = dataset["IP Address"].unique()
#stores the list pf unique ip addresses

remove_ip = []
#stores the list of ip address from where reviews have been written.

size = len(ip_list.tolist())
#stores the size of the total unique ip addresses

for i in range(size):
    # iterate through all the ip addresses
    
    brand_df = ip.get_group(ip_list[i])
    # Dataframe of brands for which reviews have been written from the same ip address
    
    brands = brand_df.groupby("product_parent")
    # grouping the products of the same brands for each ip addresses
    
    brands_list = brand_df["product_parent"].unique()
    #list of unique brands for each ip addresses
    
    no_of_brands = len(brands_list.tolist())
    # total no. of brands
    
    for j in range(no_of_brands):
        # iterate through all the brands
        
        product_df = brands.get_group(brands_list[j])
        # Dataframe of the products of each brand of each products
        
        no_of_products = len(product_df["product_id"])
        # no of products of each brand for each ip addresses
        
        if no_of_products<=2:
            # filter the reviews of the brandswith less than 3 reviews
            break
        
        indices = product_df.index.tolist()
        # indices of dataframe of products of each brand for each customers
        
        sentiment = getSentiment(product_df["review_body"][ indices[0] ])
        # sentiment of review of first product of each brand
                
        isSameSentiment = True
        
        for k in range(1,no_of_products):
            # iterate through all the reviews
            
            text = str(product_df["review_body"][indices[k]])
            # reviews of each product
            
            if getSentiment(text)!=sentiment :
                # if sentiment of 2 products of same brand are not same 
                # then check the next brand
                isSameSentiment = False
                break;
                
        if(isSameSentiment):
            # if all the sentiments are same , append ip to blocked list
            remove_ip.append(ip_list[i])
            

In [17]:
remove_ip

['193.93.167.87', '205.10.168.66']

In [53]:
'''
dataset.set_index("IP Address",inplace=True)
for ip in blocked_users:
    dataset.drop(ip,inplace=True)
dataset.reset_index()
'''

'\ndataset.set_index("IP Address",inplace=True)\nfor ip in blocked_users:\n    dataset.drop(ip,inplace=True)\ndataset.reset_index()\n'

## 4. User posting (>3) reviews on the same day with all the reviews are either positive or negative.

In [19]:
dataset.sort_values("customer_id",inplace=True)

In [20]:
dataset.head(1)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,IP Address
24496,US,10071,R1XALU24Y932WC,B0048DLA90,864204912,NEW Click Wheel Flex Cable For Apple iPod Nano...,Mobile_Electronics,5,0.0,0.0,N,Y,Five Stars,Just what I needed. Thanks.,2014-07-31,NaN


In [139]:
dataset = pd.read_csv("reviews2.csv",sep="\t")

/home/anubhav/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,4,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
#4. User posting (>3) reviews on the same day with all the reviews are either positive or negative.

customer_group = dataset.groupby("customer_id")
#creates the group of the customers 

customer_group_list = dataset["customer_id"].unique().tolist()
# list of unique customers

for i in range(len(customer_group_list)):
    # iterate through all customers , starts with 1 as column could not be included
    
    customer_reviews = customer_group.get_group( customer_group_list[i] )
    # Dataframe of data of each cutomers
    
    dates_list = customer_reviews["review_date"].unique().tolist()
    # list of dates of reviews written by each customers
    
    reviews_by_date = customer_reviews.groupby("review_date");
    # gouping reviews by date for each cutomers
    
    for j in range(len(dates_list)):
        # iterating through all dates
        
        reviews_by_date_for_pos = []
        reviews_by_date_for_neg = []

        df = reviews_by_date.get_group(dates_list[j])
        #dataframe storing the details for each details for each customers
        
        indices = df.index.tolist()
        # indices of dataframe of each date
        
        for k in range(len(df)):
            # iterating through dataframe of each day for each customers
            
            text = df["review_body"][ indices[k] ]
            #review on a single day
            
            if(getSentiment(text) == 0):
                
                #if sentiment is negative, append review_id to list of negative reviews
                reviews_by_date_for_neg.append(df["review_id"][ indices[k] ])
                
            else:
                
                #if sentiment is positive, append review_id to list of positive reviews
                reviews_by_date_for_pos.append(df["review_id"][ indices[k] ])
                
        # CONDITION FOR CONSIDERING THE FAKE REVIEW 
        
        #removing postive reviews that are written by a reviewer that are > 3 on same day
        if(len(reviews_by_date_for_pos)>3):
            remove_reviews.extend(reviews_by_date_for_pos)
        
        #removing postive reviews that are written by a reviewer that are > 3 on same day
        if(len(reviews_by_date_for_neg)>3):
            remove_reviews.extend(reviews_by_date_for_neg)
        

In [29]:
len(remove_reviews)

32348

## 5. Reviews(>3) from same IP on the same day with all the reviews are either positive or negative.

In [26]:
dataset = pd.read_csv("reviews2.csv",sep="\t")

In [30]:
#5. Reviews(>3) from same IP on the same day with all the reviews are either positive or negative.

ip_group = dataset.groupby("IP Address")
# grouping the dataset by ip addresses

ip_list = dataset["IP Address"].unique().tolist()
# stores the list of unique ip addresses

size = len(ip_list)
# total no of unique ip addresses

for i in range(size):
    # iterate through all the ip addresses
    
    reviews = ip_group.get_group( ip_list[i] )
    # dataframe of each ip
    
    dates_list = reviews["review_date"].unique().tolist()
    # list of dates of reviews by each ip addresses
    
    reviews_by_date = reviews.groupby("review_date");
    # grouping the dataframe by date
    
    for j in range(len(dates_list)):
        # iterate through all the dates
        
        reviews_by_date_for_pos = []
        reviews_by_date_for_neg = []

        reviews_for_each_day = reviews_by_date.get_group(dates_list[j])
        #dataframe of reviews for a day by each ip addresses
        
        indices = reviews_for_each_day.index.tolist()
        # list of indices of the dataframe reviews_for_each_day
        
        for k in range(len(reviews_for_each_day)):
            #iterate through all the reviews on a day by each ip addresses
            
            text = reviews_for_each_day["review_body"][ indices[k] ]
            # reviews on a day for an ip addresses
            
            if(getSentiment(text) == 0):
                
                #if sentiment is negative, append review_id to list of negative reviews
                reviews_by_date_for_neg.append(reviews_for_each_day["review_id"][ indices[k] ])
            else:
                
                #if sentiment is positive, append review_id to list of positive reviews
                reviews_by_date_for_pos.append(reviews_for_each_day["review_id"][ indices[k] ])
                
        # CONDITION FOR CONSIDERING THE FAKE REVIEW        
     
        #removing postive reviews that are written by a reviewer that are > 3 on same day
        if(len(reviews_by_date_for_pos)>3):
            remove_reviews.extend(reviews_by_date_for_pos)
        
        #removing postive reviews that are written by a reviewer that are > 3 on same day
        if(len(reviews_by_date_for_neg)>3):
            remove_reviews.extend(reviews_by_date_for_neg)
        

In [31]:
len(remove_reviews)

32356

## 6. Similar Reviews written within a range of 30 minutes

In [32]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [33]:
tfidf_vectorizer = TfidfVectorizer()

In [37]:
dataset.reset_index()
dataset.set_index("review_id")
dataset.sort_values("timestamp",inplace=True)

KeyError: 'timestamp'

In [ ]:
def OnlyStopwords(str):
    words = nltk.word_tokenize(str)
    words = [word for word in words if word not in stopwords.words("english")]
    if(len(words)==0):
        return True
    return False

In [ ]:
from nltk.corpus import wordnet

remove_reviews = []
indices = []
for i in range(len(dataset)):
    
    reviews = [str(dataset["review_body"][i])]
    
    try:
        tfidf_vectorizer.fit_transform(reviews)
    except:
        # reviews with one word and with no dictionary meaning will be invalid
        # e.g- ["c","O.K."]
        remove_reviews.append(dataset["review_id"][i]) 
        continue
        
    Time = dataset["timestamp"][i]
    # timestamp of the review that will be compared
    
    for j in range(i+1,len(dataset)):
            
        indices.append(dataset["review_id"][j])
        
        if(dataset["timestamp"][j]-Time <= 1800):
            # reviews written in 30 min of intervals will be checked for same pattern
            reviews.append(str(dataset["review_body"][j]))
        else:
            break
    
    tfidf_matrix = tfidf_vectorizer.fit_transform(reviews)
    
    #creates TF-IDF Model
    tfidf_list = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix).tolist()
    # Creates matrix based on document similarity
         
    # To check similarity b/w 2 reviews
    i_appended = False
    for k in range(1,len(tfidf_list[0])):
        #print(tfidf_list[0][k],i+k)
        
        if(tfidf_list[0][k]>0.6):
            # 0.6 is defind for the simmilarity level
            
            remove_reviews.append(dataset["review_id"][i+k])
            # i+k is to get the review id of the review
            
            if(not i_appended):
                remove_reviews.append(dataset["review_id"][i]) 
                i_appended = True
 

In [ ]:
remove_reviews

## 7. Removing reviews with no. of verbs > no. of nouns

In [4]:
dataset = pd.read_csv("reviews2.csv",sep='\t')
dataset.head(2)

/home/anubhav/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,IP Address,timestamp
0,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,217304173,BlackVue DR600GW-PMP,Mobile_Electronics,5,0.0,0.0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31,207.156.210.106,1.441005e+09
1,US,40835037,R31LOQ8JGLPRLK,B00OQMFG1Q,137313254,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5,0.0,1.0,N,Y,five star,it's great,2015-08-31,193.93.167.87,1.441038e+09


In [39]:
#7. Removing reviews with no. of verbs > no. of nouns

for i in range(len(dataset)):
    #iterate the whole dataset
    
    words = nltk.word_tokenize(str(dataset["review_body"][i]))
    #storing the words from the reviews into the list
    
    tagged_words = nltk.pos_tag(words)
    # returns list of tuples of words along with their parts of speech
    
    nouns_count = 0
    verbs_count = 0
    
    for j in range(len(tagged_words)):
        #iterate through all the words

        if(tagged_words[j][1].startswith("NN")):
            nouns_count+=1
            #counts the no. of nouns in the review

        if(tagged_words[j][1].startswith("VB")):
            verbs_count+=1
            #counts the no. of verbs in the review

    if(verbs_count>nouns_count):
        #comparing the no. of verbs and nouns
        remove_reviews.append(dataset["review_id"][i])
        #storing the review to be removed 

In [41]:
len(remove_reviews)

26131

## 8. Removing reviews with includes more first person pronouns.

In [42]:
# 8. Removing reviews with includes more first person pronouns.

for i in range(len(dataset)):
    #iterate through all the reviews
    
    dataset["review_body"][i] = str(dataset["review_body"][i]).lower()
    # converting each characters to its lower cases
    
    words = nltk.word_tokenize(dataset["review_body"][i])
    # storing the list of words for each reviews
    
    sentence = nltk.sent_tokenize(dataset["review_body"][i])
    # storing the list of sentences for each reviews
    
    count=0
    if(len(sentence)>4):
        # Checking only those reviews which have atleast 5 sentences.
        
        for j in range(len(words)):
            #iterating through all the reviews
            
            if(words[j]=="i" or words[j]=="we"):
                #counting personal pronouns
                count+=1
                
        if(count > len(sentence)/2):
            #reviews with number of personal pronouns used greater than half the no. of sentences.
            remove_reviews.append(dataset["review_id"][i])


/home/anubhav/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


KeyboardInterrupt: 

In [ ]:
remove_reviews